In [1]:
pip install mrcfile numpy matplotlib


   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ------------------------------- -------- 6.3/8.0 MB 35.1 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 31.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 30.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import mrcfile
import numpy as np
import matplotlib.pyplot as plt

# Load the MRC file
mrc_path = "emd_0008/cryoem_deposited.mrc"
with mrcfile.open(mrc_path) as mrc:
    data = mrc.data  # This is a 3D NumPy array


In [9]:
pip install scipy

   ---------------------------------------- 0.0/41.2 MB ? eta -:--:--
   -- ------------------------------------- 2.6/41.2 MB 18.9 MB/s eta 0:00:03
   --------- ------------------------------ 9.7/41.2 MB 28.8 MB/s eta 0:00:02
   ------------------- -------------------- 19.7/41.2 MB 36.5 MB/s eta 0:00:01
   -------------------------- ------------- 27.3/41.2 MB 36.8 MB/s eta 0:00:01
   ----------------------------------- ---- 36.4/41.2 MB 38.6 MB/s eta 0:00:01
   ---------------------------------------- 41.2/41.2 MB 36.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import mrcfile
import numpy as np
import scipy
# import scipy.ndimage

# Define chunk size for HR and LR
chunk_size_hr = 32
chunk_size_lr = 8
scale_factor = chunk_size_lr / chunk_size_hr  # 10/40 = 1/4

# Load original MRC file
mrc_path = "emd_0008/cryoem_deposited.mrc"
with mrcfile.open(mrc_path) as mrc:
    data = mrc.data  # This is a 3D NumPy array
    
# Compute required padding
pad_x = (chunk_size_hr - data.shape[0] % chunk_size_hr) % chunk_size_hr
pad_y = (chunk_size_hr - data.shape[1] % chunk_size_hr) % chunk_size_hr
pad_z = (chunk_size_hr - data.shape[2] % chunk_size_hr) % chunk_size_hr

# Pad with zeros
data_padded = np.pad(data, ((0, pad_x), (0, pad_y), (0, pad_z)), mode='constant')

# Get new shape
new_shape = data_padded.shape
print(f"Padded shape: {new_shape}")

# Create output folders
output_hr_dir = "hr_40"
output_lr_dir = "lr_10"
os.makedirs(output_hr_dir, exist_ok=True)
os.makedirs(output_lr_dir, exist_ok=True)

# Split into 40x40x40 cubes and generate LR
count = 0
for i in range(0, new_shape[0], chunk_size_hr):
    for j in range(0, new_shape[1], chunk_size_hr):
        for k in range(0, new_shape[2], chunk_size_hr):
            sub_volume = data_padded[i:i+chunk_size_hr, j:j+chunk_size_hr, k:k+chunk_size_hr]

            # Save HR chunk
            hr_filename = os.path.join(output_hr_dir, f"chunk_{count:03d}.mrc")
            with mrcfile.new(hr_filename, overwrite=True) as mrc_new:
                mrc_new.set_data(sub_volume.astype(np.float32))

            # Generate LR (1/4 scale)
            sub_volume_lr = scipy.ndimage.zoom(sub_volume, scale_factor, order=3)  # Bicubic-like downsampling

            # Save LR chunk
            lr_filename = os.path.join(output_lr_dir, f"chunk_{count:03d}.mrc")
            with mrcfile.new(lr_filename, overwrite=True) as mrc_new:
                mrc_new.set_data(sub_volume_lr.astype(np.float32))

            count += 1

print(f"Saved {count} HR chunks in '{output_hr_dir}'")
print(f"Saved {count} LR chunks in '{output_lr_dir}'")


Padded shape: (480, 480, 480)
Saved 1728 HR chunks in 'hr_40'
Saved 1728 LR chunks in 'lr_10'
